# Starlink: Latency over Single Days

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import matplotlib.pyplot as plt
import pycountry
import os

def country_name(country_iso):
    return pycountry.countries.get(alpha_2=country_iso).name

In [2]:
probes = pd.read_parquet("./parquet/ripe_atlas_probe_data.parquet").set_index('id')
probes

,ipv4,asn,longitude,latitude,country
id,,,,,
10743,98.97.42.182,14593,-122.9005,48.4885,US
2823,65.181.3.210,14593,145.4685,-37.9215,AU
11585,null,14593,17.9215,59.6395,SE
1297,145.224.66.114,14593,-0.9685,51.7505,GB
12863,98.97.38.104,14593,-122.7395,45.8985,US
...,...,...,...,...,...
1006246,98.97.35.40,14593,-122.3405,47.6005,US
1007280,169.155.251.77,14593,7.0915,43.8575,FR
1005562,98.97.178.117,14593,-80.3495,34.3785,US


In [3]:
tls_data = pd.read_parquet("./parquet/tls_data.parquet").join(probes, on='prb_id')
print(len(tls_data))
tls_data

312648


,af,dst_name,dst_port,src_name,method,msm_id,msm_name,prb_id,rt,ttc,timestamp,source_platform,ipv4,asn,longitude,latitude,country
0,4,atlas.ripe.net,443,180.150.31.130,TLS,14002,SSLCert,1004828,514.880933,256.740837,1664818711,RIPE ATLAS (builtin tls),206.83.112.200,14593.0,151.5885,-32.9085,AU
1,4,atlas.ripe.net,443,180.150.31.130,TLS,14002,SSLCert,1004828,510.178742,254.042678,1664861910,RIPE ATLAS (builtin tls),206.83.112.200,14593.0,151.5885,-32.9085,AU
2,4,atlas.ripe.net,443,180.150.31.130,TLS,14002,SSLCert,1004828,522.505294,259.568981,1664905110,RIPE ATLAS (builtin tls),206.83.112.200,14593.0,151.5885,-32.9085,AU
3,4,atlas.ripe.net,443,180.150.31.130,TLS,14002,SSLCert,1004828,509.919590,253.841243,1664991511,RIPE ATLAS (builtin tls),206.83.112.200,14593.0,151.5885,-32.9085,AU
4,4,atlas.ripe.net,443,180.150.31.130,TLS,14002,SSLCert,1004828,524.197979,263.020576,1664948309,RIPE ATLAS (builtin tls),206.83.112.200,14593.0,151.5885,-32.9085,AU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312643,4,atlas.ripe.net,443,145.224.67.126,TLS,14002,SSLCert,1297,99.269396,55.493388,1719181735,RIPE ATLAS (builtin tls),145.224.66.114,14593.0,-0.9685,51.7505,GB
312644,4,atlas.ripe.net,443,145.224.66.235,TLS,14002,SSLCert,1297,85.492241,38.236423,1719354534,RIPE ATLAS (builtin tls),145.224.66.114,14593.0,-0.9685,51.7505,GB
312645,4,atlas.ripe.net,443,145.224.66.184,TLS,14002,SSLCert,1297,85.246481,52.818443,1719959334,RIPE ATLAS (builtin tls),145.224.66.114,14593.0,-0.9685,51.7505,GB
312646,4,atlas.ripe.net,443,145.224.66.184,TLS,14002,SSLCert,1297,91.639314,43.911177,1720477733,RIPE ATLAS (builtin tls),145.224.66.114,14593.0,-0.9685,51.7505,GB


In [12]:
results_dir = './individual_days_results'
def filter_for(df, name, value):
    return df.loc[df[name] == value]

def filter_for_range(df, name, lower_bound, upper_bound):
    return df[(df[name] >= lower_bound) & (df[name] < upper_bound)]

def plot(hours, title, country):
    # Create a figure and axis
    fig, ax = plt.subplots()

    # Plot the boxplots
    ax.boxplot(hours, showfliers=False)

    # Set axis labels and title
    ax.set_xlabel('i-th Hour of the Day')
    ax.set_ylabel('Latencies in ms')
    ax.set_title('Boxplot of Tuples')
    plt.title(title)
    plt.savefig(results_dir + "/" + country + '/' + title.lower().replace(" ", "_") + ".pdf")
    plt.clf()

one_hour = timedelta(hours=1)
one_day = timedelta(days=1)

def plot_all_days(country):
    german_df = filter_for(tls_data, 'country', country)
    print(len(german_df))
    
    min = datetime.fromtimestamp(np.min(german_df['timestamp'].tolist()))
    max = datetime.fromtimestamp(np.max(german_df['timestamp'].tolist()))
    
    def nicify_day(d):
        return str(d.day) + "." + str(d.month) + "." + str(d.year)
    
    current = datetime(min.year, min.month, min.day)
    while current < max:
        hours = [ filter_for_range(german_df, 'timestamp', datetime.timestamp(current + (one_hour*i)), datetime.timestamp(current + (one_hour*(i+1))))['rt'] for i in range(0, 24)]
        plot(hours, "Latency Plot for " + country_name(country) +  " on " + nicify_day(current), country)
    
        current = current + one_day

if not os.path.isdir(results_dir): os.mkdir(results_dir)
countries = ['DE', 'US']
for c in countries:
    dir = os.path.join(results_dir, c)
    if not os.path.isdir(dir):
        os.mkdir(dir)
    plot_all_days(c)

25183


/tmp/ipykernel_11435/2424153751.py:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


113858


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>